------ standard imports ------ #

# Robotics, Vision & Control 3e: for Python
## Chapter 15: Vision-Based Control

In [ ]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import math
from math import pi
np.set_printoptions(linewidth=120, formatter={'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})

In [ ]:
np.random.seed(0)

In [ ]:
from machinevisiontoolbox.base import *
from machinevisiontoolbox import *
from spatialmath.base import *
from spatialmath import *

------------------------------ #

# 15.1 Position-Based Visual Servoing


In [ ]:
camera = CentralCamera.Default(pose=SE3.Trans(1, 1, -2));

In [ ]:
P = mkgrid(2, 0.5)

In [ ]:
p = camera.project_point(P, objpose=SE3.Tz(1))

In [ ]:
Te_C_G = camera.estpose(P, p, frame="camera");
Te_C_G.printline()

In [ ]:
T_Cd_G = SE3.Tz(1);

In [ ]:
T_delta = Te_C_G * T_Cd_G.inv();
T_delta.printline()

In [ ]:
camera.pose = camera.pose * T_delta.interp1(0.05);

In [ ]:
camera = CentralCamera.Default(pose = SE3.Trans(1, 1, -2));

In [ ]:
T_Cd_G = SE3.Tz(1);

In [ ]:
pbvs = PBVS(camera, P=P, pose_g=SE3.Trans(-1, -1, 2), pose_d=T_Cd_G, plotvol=[-1, 2, -1, 2, -3, 2.5])

In [ ]:
pbvs.run(200);

In [ ]:
pbvs.plot_p();     # plot image plane trajectory
pbvs.plot_vel();   # plot camera velocity
pbvs.plot_pose();  # plot camera trajectory

# 15.2 Image-Based Visual Servoing


## 15.2.1 Camera and Image Motion


In [ ]:
camera = CentralCamera.Default();

In [ ]:
P = [1, 1, 5];

In [ ]:
p0 = camera.project_point(P)

In [ ]:
p_dx = camera.project_point(P, pose=SE3.Tx(0.1))

In [ ]:
(p_dx - p0) / 0.1

In [ ]:
(camera.project_point(P, pose=SE3.Tz(0.1) ) - p0) / 0.1

In [ ]:
(camera.project_point(P, pose=SE3.Rx(0.1)) - p0) / 0.1

In [ ]:
J = camera.visjac_p(p0, depth=5)

In [ ]:
camera.flowfield([1, 0, 0, 0, 0, 0]);

In [ ]:
camera.flowfield([0, 0, 1, 0, 0, 0]);

In [ ]:
camera.flowfield([0, 0, 0, 0, 0, 1]);

In [ ]:
camera.flowfield([0, 0, 0, 0, 1, 0]);

In [ ]:
camera.visjac_p(camera.pp, depth=1)

In [ ]:
camera.f = 20e-3;
camera.flowfield([0, 0, 0, 0, 1, 0]);

In [ ]:
camera.f = 4e-3;
camera.flowfield([0, 0, 0, 0, 1, 0]);

In [ ]:
J = camera.visjac_p(camera.pp, depth=1);

In [ ]:
linalg.null_space(J)

## 15.2.2 Controlling Feature Motion


In [ ]:
camera = CentralCamera.Default(pose=SE3.Trans(1, 1, -2));

In [ ]:
P = mkgrid(2, side=0.5, pose=SE3.Tz(3));

In [ ]:
pd = 200 * np.array([[-1, -1, 1, 1], [-1, 1, 1, -1]]) + np.c_[camera.pp]

In [ ]:
p = camera.project_point(P)

In [ ]:
e = pd - p

In [ ]:
J = camera.visjac_p(p, depth=1);

In [ ]:
lmbda = 0.1;
v = lmbda * np.linalg.pinv(J) @ e.flatten(order="F")

In [ ]:
camera.pose = camera.pose @ SE3.Delta(v);

In [ ]:
camera = CentralCamera.Default(pose=SE3.Trans(1, 1, -3) * SE3.Rz(0.6));
ibvs = IBVS(camera, P=P, p_d=pd);

In [ ]:
ibvs.run(25);

In [ ]:
ibvs.plot_p();     # plot image plane trajectory
ibvs.plot_vel();   # plot camera velocity
ibvs.plot_pose();  # plot camera trajectory 

In [ ]:
ibvs.plot_jcond();

In [ ]:
%run -m IBVS-main -H

In [ ]:
out

In [ ]:
plt.plot(out.t, out.y2)

In [ ]:
plt.plot(out.clock0.t, out.clock0.x)

## 15.2.3 Estimating Feature Depth


In [ ]:
ibvs = IBVS(camera, P=P, p_d=pd, depth=1);
ibvs.run(50)
ibvs = IBVS(camera, P=P, p_d=pd, depth=10);
ibvs.run(50)

In [ ]:
ibvs = IBVS(camera, P=P, p_d=pd, depthest=True);
ibvs.run()

## 15.2.4 Performance Issues


In [ ]:
pbvs.pose_0 = SE3.Trans(-2.1, 0, -3) * SE3.Rz(5*pi/4);
pbvs.run()

In [ ]:
ibvs.pose_0 = pbvs.pose_0;
ibvs.run()
ibvs.plot_p();

In [ ]:
ibvs.pose_0 = SE3.Tz(-1) * SE3.Rz(2);
ibvs.run(50)

In [ ]:
ibvs.pose_0 = SE3.Tz(-1) * SE3.Rz(pi);
ibvs.run(10)

# 15.3 Using Other Image Features


## 15.3.1 Line Features


In [ ]:
P = circle([0, 0, 3], 0.5, resolution=3);

In [ ]:
ibvs = IBVS_l.Example(camera);  # quick problem setup
ibvs.run()

## 15.3.2 Ellipse Features


In [ ]:
P = circle([0, 0, 3], 0.5, resolution=10);

In [ ]:
p = camera.project_point(P, pose=camera.pose, retinal=True);

In [ ]:
x, y = p
A = np.column_stack([y**2, -2*x*y, 2*x, 2*y, np.ones(x.shape)]);
b = -(x**2);
E, *_ = np.linalg.lstsq(A, b, rcond=None)

In [ ]:
plane = [0, 0, 1, -3];  # plane Z=3
J = camera.visjac_e(E, plane);
J.shape

In [ ]:
ibvs = IBVS_e.Example();  # quick problem setup
ibvs.run()

## 15.3.3 Photometric Features


# 15.4 Wrapping Up


## 15.4.1 Further Reading


## 15.4.2 Exercises
